In [1]:
from kafka import KafkaConsumer
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, TimestampType
import json
import pyspark
from pyspark.sql.functions import from_json, col
from datetime import datetime

In [4]:
spark = SparkSession.builder.master("local").appName("Analytics").getOrCreate()
df = spark.read.option("header","true").csv("2022-12-06/*.csv")
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- country: string (nullable = true)
 |-- name: string (nullable = true)
 |-- number: string (nullable = true)
 |-- CVC: string (nullable = true)
 |-- expire: string (nullable = true)



In [5]:
df.count()

7

In [6]:
df.head()

Row(date='2022/12/06 16:28:00', ID='9179ce74-757a-11ed-9080-309c23ff59e3', country='Armenia', name='Lynn Riley', number='30001991975956', CVC='5867', expire='03/28')

In [39]:
df.groupBy("value").count().orderBy(col("count").desc()).limit(1).show()

+-----------+-----+
|      value|count|
+-----------+-----+
|El Salvador|    6|
+-----------+-----+

